In [ ]:
import os
import sys
import random
from langchain_community.vectorstores import Chroma
from langchain_experimental.utilities import PythonREPL
from IPython.display import display, Markdown
import jsonlines

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from complex_rag.src.document_retrieval import DocumentRetrieval
from enterprise_knowledge_retriever.src.advanced_rag_w_coding2 import CodeRAG
from enterprise_knowledge_retriever.prompts.examples import examples

CONFIG_PATH = os.path.join(kit_dir, "config.yaml")

In [ ]:
documentRetrieval = DocumentRetrieval()
*_, embedding_model_info, retrieval_info, _ = documentRetrieval.get_config_info()
embeddings = documentRetrieval.load_embedding_model()

In [ ]:
vectorstore = Chroma(
    persist_directory=kit_dir + "/data/2_parts_adi.chromadb",
    embedding_function=embeddings,
)

In [ ]:
# instantiate rag
rag = CodeRAG(
    configs=CONFIG_PATH,
    embeddings=embeddings,
    vectorstore=vectorstore,
    examples=examples,
)

# Initialize chains
rag.initialize()

# Build nodes
workflow = rag.create_rag_nodes()
print(workflow)

# Build graph
app = rag.build_rag_graph(workflow)

In [ ]:
rag.display_graph(app)

In [ ]:
def log_answers(question, output_file):
    try:
        response = rag.call_rag(app, question)
    except:
        response = {}
        response["answer"] = "Failed"

    display(Markdown("---Response---"))
    display(Markdown(response["answer"]))

    if not os.path.exists(kit_dir + "/outputs"):
        os.makedirs(kit_dir + "/outputs")
    output_dir = kit_dir + "/outputs"

    with jsonlines.open(output_dir + f"/{output_file}", "a") as writer:
        writer.write({"question": question, "response": response["answer"]})

In [ ]:
questions = []

In [ ]:
output_file = "ollama_Llama3_8B_code_router_prompt_mod.jsonl"
for question in questions:
    log_answers(
        question=question,
        output_file=output_file,
    )